# Via API e XML

- A API do Google News foi descontinuada porém existe uma versão disponível que retorna os resultados da pesquisa em formato xml

https://news.google.com/rss/search?q=marcha%20empoderamento%20salvador&hl=pt-BR&gl=BR&ceid=BR:pt-419


TODO: Verificar XML vulnerabilities
https://docs.python.org/3/library/xml.etree.elementtree.html


In [1]:
import requests
from xml.etree import ElementTree

In [2]:
url = 'https://news.google.com/rss/search?q='

In [20]:
chave_busca = ['"marcha" and "crespo" and "salvador"',
               '"marcha" and "empoderamento" and  "crespo" and "salvador"',
               '"marcha" and "empoderamento crespo" and "salvador"']

In [21]:
aux = '&hl=pt-BR&gl=BR&ceid=BR:pt-419'

In [22]:
url_final = url + chave_busca[2] + aux
url_final

'https://news.google.com/rss/search?q="marcha" and "empoderamento crespo" and "salvador"&hl=pt-BR&gl=BR&ceid=BR:pt-419'

In [23]:
response = requests.get(url_final)

In [24]:
response.status_code

200

In [25]:
tree = ElementTree.fromstring(response.content)

In [26]:
tree.tag

'rss'

In [27]:
tree.attrib

{'version': '2.0'}

In [28]:
title = tree.find('channel').findtext('title')
title

'""marcha" and "empoderamento crespo" and "salvador"" - Google Notícias'

In [29]:
tree = tree.find('channel')

In [30]:
noticias = tree.getchildren()
len(noticias)

36

In [31]:
noticias = noticias[8:]

In [32]:
len(noticias)

28

In [33]:
for noticia in noticias:
    print('    ')
    print("Titulo: " + noticia.findtext('title'))
    print("Data da Publicação: " + noticia.findtext('pubDate'))
    print("Link: " + noticia.findtext('link'))
    print("Canal: " + noticia.findtext('source'))

    
Titulo: Cultura às 14:51 Quarta edição da Marcha do Empoderamento Crespo acontece no próximo domingo Da - A Tarde On Line
Data da Publicação: Thu, 29 Nov 2018 08:00:00 GMT
Link: https://atarde.uol.com.br/cultura/noticias/2015504-quarta-edicao-da-marcha-do-empoderamento-crespo-acontece-no-proximo-domingo
Canal: A Tarde On Line
    
Titulo: É preciso encrespar! Marcha influencia cabeças no Centro da cidade - Jornal Correio
Data da Publicação: Sun, 09 Dec 2018 08:00:00 GMT
Link: https://www.correio24horas.com.br/noticia/nid/e-preciso-encrespar-marcha-influencia-cabecas-no-centro-da-cidade/
Canal: Jornal Correio
    
Titulo: Marcha do Empoderamento Crespo reúne cerca de três mil pessoas em Salvador - Correio da Bahia
Data da Publicação: Sun, 08 Nov 2015 08:00:00 GMT
Link: https://www.correio24horas.com.br/noticia/nid/marcha-do-empoderamento-crespo-reune-cerca-de-tres-mil-pessoas-em-salvador/
Canal: Correio da Bahia
    
Titulo: Marcha do Empoderamento Crespo reúne dez mil pessoas na A

## Organizando nosso script

In [ ]:
import pandas as pd
import time

In [ ]:
def add_result(result):
    df = pd.DataFrame([[
                        result.findtext('title'),
                        result.findtext('pubDate'),
                        result.findtext('link'),
                        result.findtext('source'),
                        str(time.time()).split('.')[0]]], columns=columns)

    return df

In [ ]:
def extract_results(results, results_df):
    
    for result in results:
        results_df = pd.concat([results_df, add_result(result)], ignore_index=True, sort=False)
        
    return results_df

In [ ]:
columns=['title',
         'pubDate',
         'link',
         'source', 
         'timestamp_extract']

In [ ]:
results_df = pd.DataFrame(columns=['title',
                                        'pubDate',
                                        'link',
                                        'source',
                                        'timestamp_extract',
                                  ])

In [15]:
def search_rss(string, results_df):
    
    print('Extraindo notícias para a string {0}'.format(string))
    url_final = url + string + aux
    response = requests.get(url_final)
    
    if response.status_code == 200:
        
        tree = ElementTree.fromstring(response.content)
        tree = tree.find('channel')
        news = tree.getchildren()
        news = news[8:] # As 8 primeiras posições são metadados
        
        print('Foram encontrados {0} resultados'.format(len(news)))
        
        results_df = extract_results(noticias, results_df)
    
    print('--')
    
    return results_df

In [ ]:
for string in search_strings:
    
    results_df = search_rss(string, results_df)

In [ ]:
results_df.head()